# Кластеризация. Предобработка и обучение с частичным привлечением учителя


<hr>

С.Ю. Папулин (papulin.study@yandex.ru)

### Содержание

- [Предобработка данных для задачи классификации](#Предобработка-данных-для-задачи-классификации)
- [Обучение с частичным привлечением учителя](#Обучение-с-частичным-привлечением-учителя)
- [Источники](#Источники)

Подключение библиотек:

In [ ]:
import warnings
warnings.filterwarnings('ignore') 

In [ ]:
import time
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.cluster import KMeans
from sklearn.pipeline import Pipeline

In [ ]:
import sys
sys.path.insert(0, "../lib/")
from datasets import fetch_fashion_mnist

## Предобработка данных для задачи классификации

Загрузка данных

In [ ]:
fashion_dataset = fetch_fashion_mnist(return_X_y=True)

print('Overview\n', fashion_dataset.DESCR)
print('Feature names\n', fashion_dataset.feature_names)

In [ ]:
IMAGE_INDX = 20

print('Image:')
plt.figure(figsize=[4, 4])
plt.imshow(fashion_dataset.data['train'][IMAGE_INDX].reshape(-1, 28))
plt.show()

print('Target:', fashion_dataset.target['train'][IMAGE_INDX])
print('Name:', fashion_dataset.feature_names[fashion_dataset.target['train'][IMAGE_INDX]])

Размерность данных

In [ ]:
fashion_dataset.data['train'].shape, fashion_dataset.data['test'].shape

In [ ]:
fashion_dataset.target['train'].shape, fashion_dataset.target['test'].shape

Обучающее и тестовое подмножества

In [ ]:
X_train_, X_test, y_train_, y_test = *fashion_dataset.data.values(), *fashion_dataset.target.values()

# Уменьшение количества элементов обучающего множества
X_train = X_train_[:10000]
y_train = y_train_[:10000]

# Уменьшение размера изображений
# X_train = X_train.reshape(-1, 28, 28)[:, ::2, ::2].reshape(-1, 14*14)

In [ ]:
np.unique(y_train, return_counts=True)

Обучения классификатора

In [ ]:
tick = time.time()
model = SVC(**{'C': 10, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2})
model.fit(X_train, y_train)
print("Time =", time.time() - tick)
model.score(X_test, y_test)

Формирование признаков посредством кластеризации

In [ ]:
tick = time.time()
cluster_model = KMeans(n_clusters=50, random_state=12345)
cluster_model.fit(X_train)
print("Time =", time.time() - tick)

In [ ]:
cluster_model.transform(X_train).shape

In [ ]:
# Расстояние до центров кластеров
cluster_model.transform(X_train)[:1]

In [ ]:
# Предсказание кластеров
# print(np.argmin(cluster_model.transform(X_train), axis=1)[:5])
# print(cluster_model.predict(X_train[:5]))

Классификатор с новым наборам признаков

In [ ]:
tick = time.time()
model = SVC(**{'C':10, 'kernel':'poly', 'gamma': 'scale', 'degree': 2})
model.fit(cluster_model.transform(X_train), y_train)
print('Time =', time.time() - tick)
model.score(cluster_model.transform(X_test), y_test)

Реализация посредством `Pipeline`

In [ ]:
tick = time.time()
pipeline = Pipeline([
    ('cluster_model', KMeans(n_clusters=50, random_state=12345)),
    ('classifier', SVC(**{'C':10, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2})),
])
pipeline.fit(X_train, y_train)
print('Time =', time.time() - tick)
pipeline.score(X_test, y_test)

## Обучение с частичным привлечением учителя

In [ ]:
# Везьмем первые n наблюдений
N = 50
X_train_n = X_train[:N]
y_train_n = y_train[:N]

In [ ]:
np.unique(y_train_n, return_counts=True)

In [ ]:
# Обучаем модель классификации
tick = time.time()
model = SVC(**{'C': 10, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2})
# model = LogisticRegression()
model.fit(X_train_n, y_train_n)
print('Accuracy =', model.score(X_test, y_test))
print('Time =', time.time() - tick)

In [ ]:
# Обучаем модель кластеризации
tick = time.time()
cluster_model = KMeans(n_clusters=N, random_state=12345)
cluster_model.fit(X_train)
print('Time =', time.time() - tick)

Разметка данных ближайших к кластерам

In [ ]:
# Индексы наблюдений с минимальным расстоянием до ближайщего кластера
indices = np.argmin(cluster_model.transform(X_train), axis=0)
indices

In [ ]:
cols = 10
row_num = -(-len(indices) // cols)

fig, axs = plt.subplots(row_num, cols, figsize=(14, 2*row_num), squeeze=False)
for i in range(row_num):
    for j in range(cols):
        indx = i * cols + j
        if indx >= len(indices):
            fig.delaxes(axs[i, j])
        else:
            image = X_train[indices[indx]].reshape(-1, 28)
            axs[i, j].imshow(image)
            axs[i, j].set_title(
                "cluster={}".format(indx))
            axs[i, j].axis("off")
# plt.tight_layout()
plt.show()

In [ ]:
# Массив наблюдений, соответствующих ранее полученным индексам
X_train_n_labeled = X_train[indices]

# Замечания: Эти значения должны быть внесены вручную на основе
# изображений выше. Однако здесь мы используем уже размеченный 
# набор с целевыми значениями
y_train_n_labeled = y_train[indices]

In [ ]:
np.unique(y_train_n_labeled, return_counts=True)

In [ ]:
# Обучение на новом наборе из N размеченных данных
model = SVC(**{'C': 10, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2})
model.fit(X_train_n_labeled, y_train_n_labeled)
print('Accuracy =', model.score(X_test, y_test))
print('Time =', time.time() - tick)

Разметка всего набора данных

In [ ]:
# Предсказание кластеров
с__pred = cluster_model.predict(X_train)

In [ ]:
# Сопоставим индексы кластеров и индексы классов (для размеченных вручную изображений)
y_train_labeled = y_train[indices[с__pred]]
y_train_labeled

In [ ]:
# Обучение на новых размеченных данных
tick = time.time()
model = SVC(**{'C': 10, 'kernel': 'poly', 'gamma': 'scale', 'degree': 2})
model.fit(X_train, y_train_labeled)
print('Accuracy =', model.score(X_test, y_test))
print('Time =', time.time() - tick)

Приведенные выше способы подходят для повышения качества предсказания моделей при небольшом количестве размеченных данных. Если у нас достаточно большой набор размеченных данных, то не стоит ожидать значительного увеличения качества предсказания

##  Источники

Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow by Aurélien Géron